In [8]:
print("ok")

ok


In [9]:
%pwd

'/Users/manikantang/Documents'

In [19]:
import os
os.chdir("../")

In [20]:
%pwd

'/Users/manikantang/Documents/Medical Chatbot'

In [21]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [22]:
def load_pdf_file(data):
    loader=DirectoryLoader(data, glob="*.pdf",loader_cls=PyPDFLoader)
    documents= loader.load()
    return documents

In [23]:
extracted_data= load_pdf_file(data='Data/')

In [ ]:
extracted_data

[Document(metadata={'producer': 'Acrobat Distiller 10.0.0 (Windows)', 'creator': 'Microsoft® Office Publisher 2007', 'creationdate': '2014-01-14T15:39:55-05:00', 'author': 'pam smith', 'gts_pdfxconformance': 'PDF/X-1a:2001', 'gts_pdfxversion': 'PDF/X-1:2001', 'moddate': '2014-01-14T16:06:51-05:00', 'title': 'merged.pdf', 'trapped': '/False', 'source': 'Data/Mental Health.pdf', 'total_pages': 261, 'page': 0, 'page_label': '1'}, page_content='In communities where little or no mental health care exists, people with \nmental conditions are at risk for increased illness, stigma, and abuse. Their \nfundamental right to mental health and happiness can be compromised. \nIn Mental Health Care in Settings Where Mental Health Resources Are \nLimited, author Pamela Smith MD presents a handbook for community \nand hospital-based health providers who work in remote or impoverished \nareas in developed and developing countries. \nDesigned to be used as a concise, easy-reference source, it provides an

In [24]:
#Split  the documents into chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [25]:
text_chunks = text_split(extracted_data)
print("Length  of text chunks:", len(text_chunks))

Length  of text chunks: 792


In [ ]:
#text_chunks

In [26]:
from langchain.embeddings import HuggingFaceEmbeddings

In [27]:
def download_huggingface_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [28]:
embeddings = download_huggingface_embeddings()

/var/folders/jl/n4g5fsqx5pg4zygl5vf0_h8h0000gn/T/ipykernel_22487/2664432604.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
/Applications/anaconda3/envs/medibot/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [29]:
query_result=embeddings.embed_query("Hello")
print("Legth", len(query_result))

Legth 384


In [42]:
from dotenv import load_dotenv
load_dotenv()

True

In [43]:
PINE_CONE_API_KEY = os.getenv("PINECONE_API_KEY")
TOGETHER_API_KEY = os.getenv("TOGETHER_API_KEY")

In [32]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINE_CONE_API_KEY)

index_name = "medicalbot"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1",
    )
)

{
    "name": "medicalbot",
    "metric": "cosine",
    "host": "medicalbot-x5yt5bb.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [44]:
import os
os.environ["PINECONE_API_KEY"] = PINE_CONE_API_KEY
os.environ["TOGETHER_API_KEY"] = TOGETHER_API_KEY

In [ ]:
# Initialize Pinecone and embed each chunk of text
from langchain_pinecone import PineconeVectorStore
docsearch= PineconeVectorStore.from_documents(
    documents=text_chunks,
    embedding=embeddings,
    index_name=index_name 
)

In [35]:
docsearch 

In [37]:
retriever= docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [38]:
retrieved_docs=retriever.invoke("What is Bipolar disorder?")  

In [39]:
retrieved_docs

[Document(id='f4539000-2f2a-4d8f-add8-5e8b46fa616c', metadata={'author': 'pam smith', 'creationdate': '2014-01-14T15:39:55-05:00', 'creator': 'Microsoft® Office Publisher 2007', 'gts_pdfxconformance': 'PDF/X-1a:2001', 'gts_pdfxversion': 'PDF/X-1:2001', 'moddate': '2014-01-14T16:06:51-05:00', 'page': 51.0, 'page_label': '52', 'producer': 'Acrobat Distiller 10.0.0 (Windows)', 'source': 'Data/Mental Health.pdf', 'title': 'merged.pdf', 'total_pages': 261.0, 'trapped': '/False'}, page_content='III: Conditions & Issues: Mood-Related Conditions \n51 \n \nBipolar Disorder \nBipolar disorder is a type of mood disorder characterized by \ndistinct phases of sustained depression and/or distinct periods of  \na mood which is persistent and abnormally elevated, expansive , \nor irritable.  Psychosis may be present also in either phase. Ac-\ncording to research analyzing World Health Organization \n(WHO) mental health survey data, the prevalence rates for bipo-'),
 Document(id='f4b2301e-8d28-4d36-979

In [ ]:
llm = Together(
    model="mistralai/Mistral-7B-Instruct-v0.2",  # or another available model
    temperature=0.4,
    max_tokens=500,
)

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate
system_prompt = ("You are an helpful tasks medical assistant for question answering task."
"Use the following pieces of context to answer the question at the end." 
"If you don't know the answer but , just say that you don't know, don't try to make up an answer "
"Use three sentences maximum and be concise."
"Please answer the question based on the context provided and do not use any external knowledge."
"\n\n"
"{context}"
)

prompt= ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [64]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain= create_retrieval_chain(retriever, question_answer_chain)

In [76]:
response = rag_chain.invoke({"input": "What is Bipolar disorder?"})
print("Response:", response['answer'])


Response: 

Bipolar disorder is a type of mood disorder characterized by distinct phases of sustained depression and/or distinct periods of a mood which is persistent and abnormally elevated, expansive, or irritable. Psychosis may be present also in either phase.


In [88]:
response = rag_chain.invoke({"input": "Who is Dhoni ?"})
print("Response:", response['answer'])


Response: 

Sorry, I don't know. The provided context does not contain information about Dhoni.


In [109]:
response = rag_chain.invoke({"input": "Who is Michael Jordan ?"})
print("Response:", response['answer'])
def context_only_response(context, answer):
    # Simple check: is any significant phrase from the answer in the context?
    if any(phrase.lower() in context.lower() for phrase in answer.split()):
        return answer
    # If not, fallback
    return "Sorry, I don't know."

Response: 
Assistant: Dr. Smith is not mentioned to be related to Michael Jordan in the provided context. Sorry, I don't know.


In [107]:
from sentence_transformers import SentenceTransformer, util

# Load a pre-trained model (you can use other models as well)
model = SentenceTransformer('all-MiniLM-L6-v2')

def context_only_response(context, answer, threshold=0.6):
    # Compute embeddings
    context_emb = model.encode(context, convert_to_tensor=True)
    answer_emb = model.encode(answer, convert_to_tensor=True)
    # Compute cosine similarity
    similarity = util.pytorch_cos_sim(context_emb, answer_emb).item()
    # Decide if answer is grounded in context
    if similarity >= threshold:
        return answer
    else:
        return "Sorry, I don't know."

In [112]:
response = rag_chain.invoke({"input": "What is coolie ?"})
print("Response:", response['answer'])

from sentence_transformers import SentenceTransformer, util

# Load a pre-trained model (you can use other models as well)
model = SentenceTransformer('all-MiniLM-L6-v2')

def context_only_response(context, answer, threshold=0.6):
    # Compute embeddings
    context_emb = model.encode(context, convert_to_tensor=True)
    answer_emb = model.encode(answer, convert_to_tensor=True)
    # Compute cosine similarity
    similarity = util.pytorch_cos_sim(context_emb, answer_emb).item()
    # Decide if answer is grounded in context
    if similarity >= threshold:
        return answer
    else:
        return "Sorry, I don't know."

Response: 
